In [1]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime
import os
from dotenv import load_dotenv

from scipy.signal import find_peaks

load_dotenv()
mt5_account = int(os.getenv('MT5_ACCOUNT'))
mt5_password = os.getenv('MT5_PASSWORD')
mt5_server = os.getenv('MT5_SERVER')
mt5_path = os.getenv('MT5_PATH')

if not mt5.initialize(path=mt5_path, login=mt5_account, password=mt5_password, server=mt5_server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()
print("Successfully connected to MetaTrader 5.")

symbol = "XAUUSDm"
timeframe_h4 = mt5.TIMEFRAME_H4
start_date = datetime(2023, 1, 1)
end_date = datetime.now()

h4_rates = mt5.copy_rates_range(symbol, timeframe_h4, start_date, end_date)
df_h4 = pd.DataFrame(h4_rates)
df_h4['time'] = pd.to_datetime(df_h4['time'], unit='s')
df_h4.set_index('time', inplace=True)
print(f"Fetched {len(df_h4)} H4 bars for {symbol}.")

def find_swing_points(data, lookback_candles):
    """
    Identifies the last significant swing low and swing high in an uptrend.
    
    Args:
        data (pd.DataFrame): DataFrame with 'high' and 'low' columns.
        lookback_candles (int): The number of candles on each side to determine a peak.
                               A higher number finds more significant swings.
    
    Returns:
        tuple: A tuple containing the index of the last swing low and last swing high.
               Returns (None, None) if no valid structure is found.
    """
    
    swing_highs_indices, _ = find_peaks(data['high'], distance=lookback_candles)
    swing_lows_indices, _ = find_peaks(-data['low'], distance=lookback_candles)

    if len(swing_lows_indices) == 0 or len(swing_highs_indices) == 0:
        return None, None

    last_swing_high_time = data.index[swing_highs_indices[-1]]
    last_swing_low_time = data.index[swing_lows_indices[-1]]

    if last_swing_low_time < last_swing_high_time:
        return last_swing_low_time, last_swing_high_time
    else:
        if len(swing_lows_indices) > 1:
            prev_swing_low_time = data.index[swing_lows_indices[-2]]
            relevant_highs = swing_highs_indices[data.index[swing_highs_indices] > prev_swing_low_time]
            if len(relevant_highs) > 0:
                first_relevant_high_time = data.index[relevant_highs[0]]
                return prev_swing_low_time, first_relevant_high_time

    return None, None 

last_low_time, last_high_time = find_swing_points(df_h4, 10)

if last_low_time and last_high_time:
    swing_low_price = df_h4.loc[last_low_time]['low']
    swing_high_price = df_h4.loc[last_high_time]['high']

    price_range = swing_high_price - swing_low_price

    fibo_382 = swing_high_price - (price_range * 0.382)
    fibo_618 = swing_high_price - (price_range * 0.618)
    
    fibo_1618_extension = swing_high_price + (price_range * 0.618)

    print("\n--- Fibonacci Analysis ---")
    print(f"Last Swing Low identified at: {last_low_time} -> Price: {swing_low_price:.3f}")
    print(f"Last Swing High identified at: {last_high_time} -> Price: {swing_high_price:.3f}")
    print("\nCalculated Levels:")
    print(f"  -> Golden Zone Start (38.2%): {fibo_382:.3f}")
    print(f"  -> Golden Zone End (61.8%):   {fibo_618:.3f}")
    print(f"  -> Stop Loss Level (below swing low): < {swing_low_price:.3f}")
    print(f"  -> Take Profit Target (161.8%): {fibo_1618_extension:.3f}")

else:
    print("\nCould not identify a valid swing structure in the recent data.")

mt5.shutdown()
print("\nMetaTrader 5 connection shut down.")

Successfully connected to MetaTrader 5.
Fetched 4387 H4 bars for XAUUSDm.

--- Fibonacci Analysis ---
Last Swing Low identified at: 2025-09-26 20:00:00 -> Price: 3760.050
Last Swing High identified at: 2025-09-29 04:00:00 -> Price: 3819.874

Calculated Levels:
  -> Golden Zone Start (38.2%): 3797.021
  -> Golden Zone End (61.8%):   3782.903
  -> Stop Loss Level (below swing low): < 3760.050
  -> Take Profit Target (161.8%): 3856.845

MetaTrader 5 connection shut down.
